In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool

# 3.5는 결과가 틀림
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0.1)

def plus(a, b):
    return a + b

# llm, tool, agent를 전달해야 함.
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, # 여러 개의 Input을 가진 tool을 invoke 하는 agent
    tools=[
        # 함수와 그에 대한 설명을 전달
        StructuredTool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. This tool take two arguments, both  should be numbers.",
        ),
    ],
)


# LLM은 산수를 직접 할 수 없음
prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12" # $5,273.38

# LangChain이 tool을 포함한 prompt를 만들어서 LLM에 전달함.
agent.invoke(prompt) # $4923.38



> Entering new AgentExecutor chain...
To find the total cost, I will first sum up the amounts in pairs and then add the results together.

First, I'll add $355.39 and $924.87.

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 355.39, "b": 924.87}
}
```
Observation: 1280.26
Thought:The sum of $355.39 and $924.87 is $1280.26. Next, I will add $721.2 and $1940.29.

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 721.2, "b": 1940.29}
}
```
Observation: 2661.49
Thought:The sum of $721.2 and $1940.29 is $2661.49. Next, I will add $573.63 and $65.72.

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 573.63, "b": 65.72}
}
```
Observation: 639.35
Thought:The sum of $573.63 and $65.72 is $639.35. Next, I will add $35.00 and $552.00.

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 35.00, "b": 552.00}
}
```
Observation: 587.0
Thought:The sum of $35.00 and $552.00 is $587.00. Next, I will add $76.16 and $29.12.

Action

{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost of all the items is $5273.38.'}